# ML4NLP1
## Starting Point for Exercise 1, part II

This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part II.

One of the goals of this exercise is o make you acquainted with **skorch**. You will probably need to consult the [documentation](https://skorch.readthedocs.io/en/stable/).

## Installing skorch and loading libraries

In [ ]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch'])
except ImportError:
    pass

In [1]:
%pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 740.7 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

In [2]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [3]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

## Training a classifier and making predictions

In [4]:
# download dataset
!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train
!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test
!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train
!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

zsh:1: command not found: gdown
zsh:1: command not found: gdown
zsh:1: command not found: gdown
zsh:1: command not found: gdown


In [5]:
with open(f'x_train.txt') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt') as f:
    y_test = f.read().splitlines()

In [6]:
import pandas as pd
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})

#combine x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

In [7]:
# T: Please use again the train/test data that includes English, German, Dutch, Danish, Swedish and Norwegian, plus 20 additional languages of your choice (the labels can be found in the file labels.csv)
# and adjust the train/test split if needed
data_set = pd.concat([train_df, test_df])

labels = np.unique(data_set['label'])

new_train_df = pd.DataFrame()
new_test_df = pd.DataFrame()
for label in labels:
    label_instances = data_set[data_set['label'] == label]
    train_instances = label_instances.iloc[:800]
    test_instances = label_instances.iloc[800:]
    new_train_df = new_train_df.append(train_instances)
    new_test_df = new_test_df.append(test_instances)

train_df = new_train_df.reset_index(drop=True)
test_df = new_test_df.reset_index(drop=True)    

subset_labels = {'eng', 'deu', 'nld', 'dan', 'swe', 'nob'}
other_labels = set(labels) - subset_labels
subset_labels = subset_labels | set(list(other_labels)[:20])

sub_train_set = train_df[train_df['label'].isin(subset_labels)]
sub_test_set = test_df[test_df['label'].isin(subset_labels)]
len(sub_train_set), len(sub_test_set)

(20800, 5200)

In [44]:
# T: use your adjusted code to encode the labels here
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
le_fitted = label_encoder.fit(sub_train_set['label'])
y_train_dev, y_test = le_fitted.transform(sub_train_set['label']), le_fitted.transform(sub_test_set['label'])

In [ ]:
# T: In the following, you can find a small (almost) working example of a neural network. Unfortunately, again, the cat messed up some of the code. Please fix the code such that it is executable.

In [41]:
# First, we extract some simple features as input for the neural network
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True)
X = vectorizer.fit_transform(sub_train_set['text'].to_numpy())

In [10]:
len(le_fitted.classes_)

26

In [42]:
X = X.astype(np.float32)
y = y_train_dev.astype(np.int64)

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [47]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=200,
            nonlin=F.relu,
            in_features=100,
            out_classes = 26
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.in_features = in_features
        self.out_classes = out_classes

        self.dense0 = nn.Linear(in_features, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 50)
        self.output = nn.Linear(50, out_classes)

    def forward(self, X, **kwargs):
      X = self.nonlin(self.dense0(X))
      X = F.relu(self.dense1(X))
      X = self.output(X)
      return X.squeeze(dim=1)

In [48]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=500,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    verbose = 1,
    # device='cuda',  # comment this to train with CPU
)

Here we use the default hyperparameters. After 500 epochs, the validation accuracy is still under 80%.

In [49]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.8255       0.0385        5.9435  1.4034
      2        2.7790       0.0661        3.8403  1.2888
      3        2.3588       0.0685        7.4595  1.2297
      4        1.7382       0.0690        6.5984  1.1848
      5        1.4058       0.0700        7.1200  1.1891
      6        1.3850       0.0704        7.2401  1.1937
      7        1.2767       0.0707        7.6603  1.2153
      8        1.2406       0.0714        7.5855  1.2613
      9        1.2342       0.0986        7.5546  1.1961
     10        1.2355       0.1019        7.4538  1.1732
     11        1.1970       0.1060        6.7393  1.1931
     12        1.1454       0.1065        5.8465  1.1922
     13        1.1357       0.1055        5.8824  1.2132
     14        1.0472       0.1065        6.0680  1.2427
     15        1.0091       0.1082        6.7090  1.3529
     16        1.0354       0.1

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=100, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=26, bias=True)
  ),
)

In [55]:
from sklearn.metrics import accuracy_score

y_pred = net.predict(X)
print("Training accuracy:", accuracy_score(y_true=y_train_dev, y_pred=y_pred))

X_test = vectorizer.fit_transform(sub_test_set['text'].to_numpy())
X_test = X_test.astype(np.float32)
y_pred = net.predict(X_test)

print("Testing accuracy:", accuracy_score(y_true=y_test, y_pred=y_pred))

Training accuracy: 0.8454807692307692
Testing accuracy: 0.24826923076923077


The testing accuracy is too small, which means that there may be an overfitting. 

## GridSearchCV: find best params

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from skorch.callbacks import EarlyStopping
from sklearn.preprocessing import FunctionTransformer


vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True)

def to_float32(X):
    return X.astype(np.float32)

# Here we make a pipeline as our model. 
# The 'to_float32' is used to transform the output of vectorizer to float 32, else there will be error.
text_NN = Pipeline([
    ('CV', CountVectorizer(max_features=100, binary=True)),
    ('to_float32', FunctionTransformer(to_float32)),
    ('NN', NeuralNetClassifier(
    ClassifierModule,
    callbacks = [EarlyStopping()],
    max_epochs=1000,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    verbose = 1,
))
])

# Here are the parameters we want to find best.
# These include the 5 requirements in Exercise 1 document. 
param_grid = {  # layer sizes
                'NN__module__num_units': [200, 300], 

                # activation function
                'NN__module__nonlin': [nn.ReLU(), nn.LeakyReLU()],  

                # solvers
                'NN__optimizer__lr': [0.1, 0.01],

                # early stopping
                'NN__callbacks__EarlyStopping__patience': [10, 20], 

                # vect params 
                'CV__ngram_range': [(1, 1), (2, 2)], 
    }


In [15]:
# We use 3 cross-validation sets.
gs_NN = GridSearchCV(text_NN, param_grid, cv=3, verbose=1)


In [16]:
# For saving GridSearchCV time and improving the efficiency, 
# we sample 600 instances from the sub traning set,
# make it a new dataset.
train_gs = sub_train_set.sample(600, random_state=2)
# X_gs = vectorizer.fit_transform(train_gs['text'].to_numpy())
X_gs = train_gs['text']
y_gs = le_fitted.transform(train_gs['label'])


# X_gs = X_gs.astype(np.float32)
y_gs = y_gs.astype(np.int64)

In [17]:
gs_NN.fit(X_gs, y_gs)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.2609       0.0375        3.2562  0.2988
      2        3.2574       0.0500        3.2536  0.0270
      3        3.2541       0.0500        3.2511  0.0253
      4        3.2508       0.0625        3.2486  0.0296
      5        3.2475       0.0750        3.2460  0.0268
      6        3.2442       0.0750        3.2435  0.0255
      7        3.2409       0.0875        3.2409  0.0262
      8        3.2376       0.0875        3.2383  0.0258
      9        3.2342       0.0875        3.2356  0.0254
     10        3.2308       0.0875        3.2330  0.0267
     11        3.2273       0.0875        3.2302  0.0299
     12        3.2238       0.0625        3.2274  0.0313
     13        3.2202       0.0625        3.2245  0.0251
     14        3.2165       0.0500        3.2215  0.0267
     15        3.2127      

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CV',
                                        CountVectorizer(binary=True,
                                                        max_features=100)),
                                       ('to_float32',
                                        FunctionTransformer(func=<function to_float32 at 0x7fa49d9f2290>)),
                                       ('NN',
                                        <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
))]),
             param_grid={'CV__ngram_range': [(1, 1), (2, 2)],
                         'NN__callbacks__EarlyStopping__patience': [10, 20],
                         'NN__module__nonlin': [ReLU(),
                                                LeakyReLU(negative_slope=0.01)],
                         'NN__module__num_units': [200, 300],
                         'NN__optimizer__lr': [0.1, 0.01]},
             verbose=1)

Print out the first 5 best params. 

In [18]:
NN_df = pd.DataFrame.from_dict(gs_NN.cv_results_)
NN_df.sort_values(by=["rank_test_score"]).iloc[:5]

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
14      20.673249      8.604458         0.031165        0.004644   
4        9.996614      1.474691         0.020927        0.001210   
2        9.987646      1.741863         0.044477        0.011916   
0        9.142182      1.228296         0.019976        0.000925   
12      16.889552      4.308202         0.109106        0.118992   

   param_CV__ngram_range param_NN__callbacks__EarlyStopping__patience  \
14                (1, 1)                                           20   
4                 (1, 1)                                           10   
2                 (1, 1)                                           10   
0                 (1, 1)                                           10   
12                (1, 1)                                           20   

          param_NN__module__nonlin param_NN__module__num_units  \
14  LeakyReLU(negative_slope=0.01)                         300   
4   LeakyReLU(negative_slope=0.01)                         200   
2                           ReLU()                         300   
0                           ReLU()                         200   
12  LeakyReLU(negative_slope=0.01)                         200   

   param_NN__optimizer__lr                                             params  \
14                     0.1  {'CV__ngram_range': (1, 1), 'NN__callbacks__Ea...   
4                      0.1  {'CV__ngram_range': (1, 1), 'NN__callbacks__Ea...   
2                      0.1  {'CV__ngram_range': (1, 1), 'NN__callbacks__Ea...   
0                      0.1  {'CV__ngram_range': (1, 1), 'NN__callbacks__Ea...   
12                     0.1  {'CV__ngram_range': (1, 1), 'NN__callbacks__Ea...   

    split0_test_score  split1_test_score  split2_test_score  mean_test_score  \
14              0.675              0.705              0.735         0.705000   
4               0.700              0.700              0.715         0.705000   
2               0.685              0.700              0.725         0.703333   
0               0.690              0.690              0.725         0.701667   
12              0.675              0.710              0.720         0.701667   

    std_test_score  rank_test_score  
14        0.024495                1  
4         0.007071                1  
2         0.016499                3  
0         0.016499                4  
12        0.019293                4

Print out the best params. 

In [19]:
NN_df.sort_values(by=["rank_test_score"]).iloc[0]['params']

{'CV__ngram_range': (1, 1),
 'NN__callbacks__EarlyStopping__patience': 20,
 'NN__module__nonlin': LeakyReLU(negative_slope=0.01),
 'NN__module__num_units': 300,
 'NN__optimizer__lr': 0.1}

## Continue Improving the Neural Network

Using the params we got from GridSearchCV, we explore if increasing the number of features will make the accuracy higher. Here this parameter is both max_features for CV and in_features for NN (they should be the same).

We test features in [100, 200, 500, 5000]. 
As we increase the number, the model performs better.

Model with features = 100 has test accuracy around 0.76 < 0.8
Model with features = 200, 500, 5000 has test accuracy around 0.83, 0.89, 0.96, all higher than 0.8. 

### Features = 100

In [32]:
best_model = Pipeline([
    ('CV', CountVectorizer(max_features=100, binary=True, ngram_range=(1, 1))),
    ('to_float32', FunctionTransformer(to_float32)),
    ('NN', NeuralNetClassifier(
    ClassifierModule(nonlin=nn.LeakyReLU(negative_slope=0.01), num_units=300, in_features=100),
    callbacks = [EarlyStopping(patience=20)],
    max_epochs=1500,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    verbose = 1,
))
])



In [33]:
best_model.fit(sub_train_set['text'], y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.0741       0.0385        3.4108  1.3251
      2        2.5170       0.0385        4.5382  1.4787
      3        1.6888       0.0387        4.5461  1.2600
      4        1.1648       0.0594        4.1666  1.3007
      5        0.9419       0.1062        3.6343  1.3249
      6        0.7584       0.1481        3.1380  1.2892
      7        0.6502       0.2038        2.7091  1.4012
      8        0.6305       0.3257        2.4944  1.3116
      9        0.6235       0.3945        2.1699  1.2160
     10        0.5580       0.4370        1.9848  1.2959
     11        0.5559       0.4623        1.8276  1.2877
     12        0.5421       0.4805        1.6881  1.2552
     13        0.5145       0.4911        1.6216  2.2428
     14        0.5095       0.5212        1.6078  1.2832
     15        0.5066       0.5312        1.6426  1.2618
     16        0.5015       0.5

Pipeline(steps=[('CV', CountVectorizer(binary=True, max_features=100)),
                ('to_float32',
                 FunctionTransformer(func=<function to_float32 at 0x7fa49d9f2290>)),
                ('NN',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (nonlin): LeakyReLU(negative_slope=0.01)
    (dense0): Linear(in_features=100, out_features=300, bias=True)
    (dense1): Linear(in_features=300, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=26, bias=True)
  ),
))])

In [34]:

y_pred = best_model.predict(sub_test_set['text'])

from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_test, y_pred=y_pred)

0.7586538461538461

### Features = 200

In [25]:
best_model = Pipeline([
    ('CV', CountVectorizer(max_features=200, binary=True, ngram_range=(1, 1))),
    ('to_float32', FunctionTransformer(to_float32)),
    ('NN', NeuralNetClassifier(
    ClassifierModule(nonlin=nn.LeakyReLU(negative_slope=0.01), num_units=300, in_features=200),
    callbacks = [EarlyStopping(patience=20)],
    max_epochs=1500,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    verbose = 1,
))
])



In [26]:
best_model.fit(sub_train_set['text'], y)


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.1673       0.0490        3.3025  1.5055
      2        2.7013       0.0385        4.1932  1.4031
      3        1.9258       0.0385        4.8715  1.4326
      4        1.2537       0.0385        4.8225  1.4096
      5        0.8950       0.0442        4.4507  1.4068
      6        0.7137       0.0832        3.8319  1.4306
      7        0.5988       0.1490        3.3293  1.3824
      8        0.5096       0.2928        2.9337  1.3604
      9        0.4774       0.3380        2.6013  1.3466
     10        0.4447       0.4219        2.1665  1.3489
     11        0.3770       0.5240        1.9000  1.3424
     12        0.3453       0.5805        1.7161  1.3111
     13        0.3321       0.6024        1.5712  1.3603
     14        0.3283       0.6248        1.4213  1.3307
     15        0.3212       0.6490        1.3134  1.3342
     16        0.3242       0.6

Pipeline(steps=[('CV', CountVectorizer(binary=True, max_features=200)),
                ('to_float32',
                 FunctionTransformer(func=<function to_float32 at 0x7fa49d9f2290>)),
                ('NN',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (nonlin): LeakyReLU(negative_slope=0.01)
    (dense0): Linear(in_features=200, out_features=300, bias=True)
    (dense1): Linear(in_features=300, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=26, bias=True)
  ),
))])

In [27]:

y_pred = best_model.predict(sub_test_set['text'])

from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_test, y_pred=y_pred)

0.8271153846153846

### Features = 500

In [28]:
best_model = Pipeline([
    ('CV', CountVectorizer(max_features=500, binary=True, ngram_range=(1, 1))),
    ('to_float32', FunctionTransformer(to_float32)),
    ('NN', NeuralNetClassifier(
    ClassifierModule(nonlin=nn.LeakyReLU(negative_slope=0.01), num_units=300, in_features=500),
    callbacks = [EarlyStopping(patience=20)],
    max_epochs=1500,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    verbose = 1,
))
])

best_model.fit(sub_train_set['text'], y)


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.2169       0.0483        3.2522  1.5038
      2        2.9192       0.0385        3.6267  1.6627
      3        2.1892       0.0385        4.4629  1.6148
      4        1.4717       0.0418        4.4073  1.5156
      5        0.9689       0.0685        3.7824  1.6426
      6        0.6851       0.1385        3.0717  1.5657
      7        0.5236       0.2642        2.7214  1.5109
      8        0.4803       0.3683        2.1691  1.5270
      9        0.3942       0.5031        1.7227  1.6124
     10        0.3312       0.5916        1.4371  1.6260
     11        0.3121       0.6392        1.2936  1.8306
     12        0.3053       0.6813        1.1093  1.5649
     13        0.2676       0.7127        0.9558  1.6118
     14        0.2428       0.7361        0.8682  1.5398
     15        0.2293       0.7512        0.8219  1.6543
     16        0.2213       0.7

Pipeline(steps=[('CV', CountVectorizer(binary=True, max_features=500)),
                ('to_float32',
                 FunctionTransformer(func=<function to_float32 at 0x7fa49d9f2290>)),
                ('NN',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (nonlin): LeakyReLU(negative_slope=0.01)
    (dense0): Linear(in_features=500, out_features=300, bias=True)
    (dense1): Linear(in_features=300, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=26, bias=True)
  ),
))])

In [29]:

y_pred = best_model.predict(sub_test_set['text'])

from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_test, y_pred=y_pred)

0.8915384615384615

### Features = 5000

In [30]:
best_model = Pipeline([
    ('CV', CountVectorizer(max_features=5000, binary=True, ngram_range=(1, 1))),
    ('to_float32', FunctionTransformer(to_float32)),
    ('NN', NeuralNetClassifier(
    ClassifierModule(nonlin=nn.LeakyReLU(negative_slope=0.01), num_units=300, in_features=5000),
    callbacks = [EarlyStopping(patience=20)],
    max_epochs=1500,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    verbose = 1,
))
])

best_model.fit(sub_train_set['text'], y)



  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.2398       0.1233        3.2459  4.2208
      2        2.9655       0.0385        3.5212  4.2504
      3        2.2970       0.0639        4.6039  4.2599
      4        1.4144       0.0827        4.5936  4.4118
      5        0.8222       0.1416        3.7297  4.2921
      6        0.5442       0.2466        2.8948  4.2670
      7        0.4255       0.3224        2.4504  4.5198
      8        0.3679       0.4305        1.9979  4.2895
      9        0.3228       0.5505        1.5345  4.2424
     10        0.2656       0.6377        1.1196  4.2365
     11        0.2089       0.7041        0.8835  4.2500
     12        0.1804       0.7522        0.7441  4.2944
     13        0.1620       0.7827        0.6529  4.5339
     14        0.1480       0.8036        0.5871  4.2106
     15        0.1363       0.8236        0.5376  4.2780
     16        0.1262       0.8

Pipeline(steps=[('CV', CountVectorizer(binary=True, max_features=5000)),
                ('to_float32',
                 FunctionTransformer(func=<function to_float32 at 0x7fa49d9f2290>)),
                ('NN',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (nonlin): LeakyReLU(negative_slope=0.01)
    (dense0): Linear(in_features=5000, out_features=300, bias=True)
    (dense1): Linear(in_features=300, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=26, bias=True)
  ),
))])

In [31]:

y_pred = best_model.predict(sub_test_set['text'])

from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_test, y_pred=y_pred)

0.9626923076923077